In [ ]:
from __future__ import absolute_import, division, print_function

# Import TensorFlow >= 1.10 and enable eager execution
import tensorflow as tf

tf.enable_eager_execution()

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

import unicodedata
import re
import numpy as np
import os
import time

print(tf.__version__)

1.15.0-rc3


In [ ]:
# Code to read csv file into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
link = 'https://drive.google.com/open?id=1lh4phSO2d1qmjWENgIaGdcwpZzYB5i0M'

In [ ]:
fluff, id = link.split('=')

In [ ]:
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('Filename.txt')  
df = open('Filename.txt', 'rb').read().decode(encoding='utf-8')
# Dataset is now stored in a Pandas Dataframe

In [ ]:
df = df.split()
df[0] = "khushboo";

In [ ]:
def preprocessWord(list):
  modifList = []
  for w in list:
    w = "$"+w+"@"
    modifList.append(w)
  return modifList
  

In [ ]:
input_text = preprocessWord(df[0::2])
target_text = preprocessWord(df[1::2])
input_temp = []
target_temp = []
for i in range(len(input_text)):
  if(len(input_text[i]) <= 25):
    input_temp.append(input_text[i])
    target_temp.append(target_text[i])
#     print(target_text[i],len(target_text[i]))
input_text = input_temp
target_text = target_temp

In [ ]:
input_vocab = sorted(set("".join(input_text)))
target_vocab = sorted(set("".join(target_text)))

In [ ]:
# Creating a mapping from unique characters to indices in input text English
char2idxEn = {}
idx2charEn = {}
char2idxEn["<"] = 0
for index, word in enumerate(input_vocab):
      char2idxEn[word] = index + 1
# print(char2idxEn)
for word, index in char2idxEn.items():
      idx2charEn[index] = word


# Creating a mapping from unique characters to indices in input text Hindi
char2idxHi = {}
idx2charHi = {}
char2idxHi["<"] = 0
for index, word in enumerate(target_vocab):
      char2idxHi[word] = index + 1
for word, index in char2idxHi.items():
      idx2charHi[index] = word

In [ ]:
def getIpTextasInt(c):
  f=np.array([ char2idxEn[i] for i in c])
  return f

def getTargTextasInt(c):
  f=np.array([ char2idxHi[i] for i in c])
  return f

In [ ]:
input_text_as_int = np.array([getIpTextasInt(c) for c in input_text])
input_char_dataset = [tf.data.Dataset.from_tensor_slices(i) for i in input_text_as_int]

In [ ]:
target_text_as_int = np.array([getTargTextasInt(c) for c in target_text])
target_char_dataset = [tf.data.Dataset.from_tensor_slices(i) for i in target_text_as_int]


In [ ]:
examples_per_epoch = len(input_text)

In [ ]:
BATCH_SIZE = 64
steps_per_epoch = examples_per_epoch//BATCH_SIZE
BUFFER_SIZE = 10000

steps_per_epoch

481

In [ ]:
def max_length(tensor):
    return max(len(t) for t in tensor)

In [ ]:
input_tensor = [];
target_tensor = [];
for i in range(0, len(input_text)):
  input_tensor.append(tf.convert_to_tensor(input_text_as_int[i]))
  target_tensor.append(tf.convert_to_tensor(target_text_as_int[i]))
maxlen_inp = max_length(input_tensor);
maxlen_targ = max_length(target_tensor);
input_tensor = tf.keras.preprocessing.sequence.pad_sequences(input_tensor, padding='post', maxlen = maxlen_inp)
# //why padding in target
target_tensor = tf.keras.preprocessing.sequence.pad_sequences(target_tensor, padding='post', maxlen = maxlen_targ)
# input_tensor = tf.convert_to_tensor(input_tensor)
# target_tensor = tf.convert_to_tensor(target_tensor)

In [ ]:
# Creating training and validation sets using an 80-20 split
input_tensor_train, input_tensor_val, target_tensor_train, target_tensor_val = train_test_split(input_tensor, target_tensor, test_size=0.2)

# Show length
len(input_tensor_train), len(target_tensor_train), len(input_tensor_val), len(target_tensor_val)

(24658, 24658, 6165, 6165)

In [ ]:
BUFFER_SIZE = len(input_tensor)
BATCH_SIZE = 64
N_BATCH = BUFFER_SIZE//BATCH_SIZE
embedding_dim = 300
units = 1500
input_vocab_size = len(input_vocab)
target_vocab_size = len(target_vocab)
dataset = tf.data.Dataset.from_tensor_slices((input_tensor_train, target_tensor_train)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

In [ ]:
dataset

<DatasetV1Adapter shapes: ((64, 23), (64, 20)), types: (tf.int32, tf.int32)>

In [ ]:
def gru(units):
  # If you have a GPU, we recommend using CuDNNGRU(provides a 3x speedup than GRU)
  # the code automatically does that.
  if tf.test.is_gpu_available():
    return tf.keras.layers.CuDNNGRU(units, 
                                    return_sequences=True, 
                                    return_state=True, 
                                    recurrent_initializer='glorot_uniform')
  else:
    return tf.keras.layers.GRU(units, 
                               return_sequences=True, 
                               return_state=True, 
                               recurrent_activation='sigmoid', 
                               recurrent_initializer='glorot_uniform')

In [ ]:
class Encoder(tf.keras.Model):
    def __init__(self, input_vocab_size, embedding_dim, enc_units, batch_sz):
        super(Encoder, self).__init__()
        self.batch_sz = batch_sz
        self.enc_units = enc_units
        self.embedding = tf.keras.layers.Embedding(input_vocab_size, embedding_dim)
        self.gru = gru(self.enc_units)
        
    def call(self, x, hidden):
#         print(x)
        x = self.embedding(x)
#         print(x)
        output, state = self.gru(x, initial_state = hidden)        
        return output, state
    
    def initialize_hidden_state(self):
        return tf.zeros((self.batch_sz, self.enc_units))

In [ ]:
# why fixed length encoder and decoder ? 
# why decoder is not predicting padding characters?

In [ ]:
class Decoder(tf.keras.Model):
    def __init__(self, target_vocab_size, embedding_dim, dec_units, batch_sz):
        super(Decoder, self).__init__()
        self.batch_sz = batch_sz
        self.dec_units = dec_units
        self.embedding = tf.keras.layers.Embedding(target_vocab_size, embedding_dim,trainable=True)
        self.gru = gru(self.dec_units)
        self.fc = tf.keras.layers.Dense(target_vocab_size)
        
        # used for attention
        self.W1 = tf.keras.layers.Dense(self.dec_units)
        self.W2 = tf.keras.layers.Dense(self.dec_units)
        self.V = tf.keras.layers.Dense(1)
        
    def call(self, x, hidden, enc_output):
        # enc_output shape == (batch_size, max_length, hidden_size)
        
        # hidden shape == (batch_size, hidden size)
        # hidden_with_time_axis shape == (batch_size, 1, hidden size)
        # we are doing this to perform addition to calculate the score
        hidden_with_time_axis = tf.expand_dims(hidden, 1)  
        
        # score shape == (batch_size, max_length, 1)
        # we get  1 at the last axis because we are applying tanh(FC(EO) + FC(H)) to self.V
#         check this area -------------------------------------- and check the input you are giving 
        score = self.V(tf.nn.tanh(self.W1(enc_output) + self.W2(hidden_with_time_axis)))
#         print("hii")
#         print(self.W1(enc_output).shape)
#         print(self.W2(hidden_with_time_axis).shape)
#         print(score.shape)
        # attention_weights shape == (batch_size, max_length, 1)
        attention_weights = tf.nn.softmax(score, axis=1)
#         print(attention_weights[1])
        
        # context_vector shape after sum == (batch_size, hidden_size)
        context_vector = attention_weights * enc_output
        context_vector = tf.reduce_sum(context_vector, axis=1)
        
#         print(context_vector.shape)
        
        # x shape after passing through embedding == (batch_size, 1, embedding_dim)
        x = self.embedding(x)
#         print(x);
        # x shape after concatenation == (batch_size, 1, embedding_dim + hidden_size)
        x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)
#         print(context_vector);
        
        # passing the concatenated vector to the GRU
        output, state = self.gru(x)
        
        # output shape == (batch_size * 1, hidden_size)
        output = tf.reshape(output, (-1, output.shape[2]))
        
        # output shape == (batch_size * 1, vocab)
        x = self.fc(output)
        
        return x, state, attention_weights
        
    def initialize_hidden_state(self):
        return tf.zeros((self.batch_sz, self.dec_units))

In [ ]:
encoder = Encoder(input_vocab_size+1, embedding_dim, units, BATCH_SIZE)
decoder = Decoder(target_vocab_size+1, embedding_dim, units, BATCH_SIZE)

In [ ]:
optimizer = tf.train.AdamOptimizer()


def loss_function(real, pred):
  mask = 1 - np.equal(real, 0)
  loss_ = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=real, logits=pred) * mask
  return tf.reduce_mean(loss_)

In [ ]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(optimizer=optimizer,
                                 encoder=encoder,
                                 decoder=decoder)

In [ ]:
EPOCHS = 25

for epoch in range(EPOCHS):
    start = time.time()
    
    hidden = encoder.initialize_hidden_state()
    total_loss = 0
    
    for (batch, (inp, targ)) in enumerate(dataset):
        loss = 0
        with tf.GradientTape() as tape:
            enc_output, enc_hidden = encoder(inp, hidden)
            dec_hidden = enc_hidden
            dec_input = tf.expand_dims([char2idxHi['$']] * BATCH_SIZE, 1)       
            
            # Teacher forcing - feeding the target as the next input
            for t in range(1, targ.shape[1]):
                # passing enc_output to the decoder
                predictions, dec_hidden, _ = decoder(dec_input, dec_hidden, enc_output)

                loss += loss_function(targ[:, t], predictions)
                # using teacher forcing
                dec_input = tf.expand_dims(targ[:, t], 1)
        
        batch_loss = (loss / int(targ.shape[1]))
        
        total_loss += batch_loss
        
        variables = encoder.variables + decoder.variables
        
        gradients = tape.gradient(loss, variables)
        
        optimizer.apply_gradients(zip(gradients, variables))
        if batch % 100 == 0:
            print('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1,
                                                         batch,
                                                         batch_loss.numpy()))
    print('Epoch {} Loss {:.4f}'.format(epoch + 1,
                                        total_loss / N_BATCH))
    print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

Epoch 1 Batch 0 Loss 1.4157
Epoch 1 Batch 100 Loss 0.8034
Epoch 1 Batch 200 Loss 0.7076
Epoch 1 Batch 300 Loss 0.2909
Epoch 1 Loss 0.5065
Time taken for 1 epoch 272.55521535873413 sec

Epoch 2 Batch 0 Loss 0.2028
Epoch 2 Batch 100 Loss 0.1212
Epoch 2 Batch 200 Loss 0.0986
Epoch 2 Batch 300 Loss 0.1287
Epoch 2 Loss 0.0991
Time taken for 1 epoch 267.90004682540894 sec

Epoch 3 Batch 0 Loss 0.1275
Epoch 3 Batch 100 Loss 0.0685
Epoch 3 Batch 200 Loss 0.0916
Epoch 3 Batch 300 Loss 0.1261
Epoch 3 Loss 0.0858
Time taken for 1 epoch 271.29715275764465 sec

Epoch 4 Batch 0 Loss 0.1127
Epoch 4 Batch 100 Loss 0.0662
Epoch 4 Batch 200 Loss 0.0831
Epoch 4 Batch 300 Loss 0.1156
Epoch 4 Loss 0.0716
Time taken for 1 epoch 272.5895211696625 sec

Epoch 5 Batch 0 Loss 0.1023
Epoch 5 Batch 100 Loss 0.0586
Epoch 5 Batch 200 Loss 0.0746
Epoch 5 Batch 300 Loss 0.1028
Epoch 5 Loss 0.0650
Time taken for 1 epoch 271.3683776855469 sec



In [ ]:
##beam search

In [ ]:
def evaluate(word, encoder, decoder,max_length_inp, max_length_targ):
    attention_plot = np.zeros((max_length_targ, max_length_inp))
    inputs = [char2idxEn[i] for i in (word)]
    inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs], maxlen=max_length_inp, padding='post')
    inputs = tf.convert_to_tensor(inputs)
    # print(inputs)
    setInit = {};
    setProb = [];
    tempsetInit = [];
    result = ''
    hidden = [tf.zeros((1, units))]
    enc_out, enc_hidden = encoder(inputs, hidden)
    
#     print(enc_hidden)
#     print(enc_out)
#     print(enc_out)
    dec_hidden = enc_hidden
    dec_input = tf.expand_dims([char2idxHi['$']], 0)
    
    arr = [];
    for t in range(max_length_targ-1):
#         print(dec_input);
        predictions, dec_hidden, attention_weights = decoder(dec_input, dec_hidden, enc_out)
#         print(predictions)
        # storing the attention weights to plot later on
        attention_weights = tf.reshape(attention_weights, (-1, ))
        attention_plot[t] = attention_weights.numpy()

        [values, predicted_id] = tf.math.top_k( predictions[0]).numpy(),k=3,sorted=True,name=None)
        if(len(setInit) == 0):
            setInit[str(predicted_id[0])] = values[0];
            setInit[str(predicted_id[1])] = values[1];
            setInit[str(predicted_id[2])] = values[2];
        
        else:
          for i in setInit :

#         print(predictions[0].numpy)
#         print(tf.argmax(predictions[0]).numpy())
        
        arr.append(predicted_id);
        if idx2charHi[predicted_id] == '@':
            return result, word, attention_plot, arr
        result += idx2charHi[predicted_id]
        # the predicted ID is fed back into the model
        dec_input = tf.expand_dims([predicted_id], 0)

    return result, word, attention_plot, arr

In [ ]:
# function for plotting the attention weights
def plot_attention(attention, word, predicted_word):
    fig = plt.figure(figsize=(10,10))
    ax = fig.add_subplot(1, 1, 1)
    ax.matshow(attention, cmap='viridis')
    
    fontdict = {'fontsize': 14}
    
    ax.set_xticklabels([''] + word, fontdict=fontdict, rotation=90)
    ax.set_yticklabels([''] + predicted_word, fontdict=fontdict)

    plt.show()

In [ ]:
def translate(word, encoder, decoder, max_length_inp, max_length_targ):
    print(word);
    result, word, attention_plot, arr = evaluate(word, encoder, decoder, max_length_inp, max_length_targ)
    
    print('Input: {}'.format(word))
    print('Predicted translation: {}'.format(result))

In [ ]:
def predictAccuracy(encoder,decoder,maxlen_inp,maxlen_targ,mismatch_pair,strmismatch):
  ip = "";
  cnt = 0;
  output = "";
  trueVal = "";
  cnt2char = 0;
  totalcharcnt = 0;
  avgmismatch = 0;
  mismatchcnt = 0;
  l = len(input_tensor_val);
  for i in range(0, l):
    # print(100*i/l, "% completed")
    word = input_tensor_val[i];
    for r in word:
      if(r == 0):
        break
      ip += idx2charEn[r]
    result, word, attention_plot, arr = evaluate(ip, encoder, decoder, maxlen_inp, maxlen_targ)
    
    for r in arr:
      output += idx2charHi[r];

    word = target_tensor_val[i];
    # print(word)
    for r in word[1:]:
      if(r == 0):
        break
      trueVal += idx2charHi[r]
    if(len(output) == len(trueVal)):
      wordlen = len(trueVal);
      totalcharcnt += int(wordlen);
      tempcnt = 0;
      for k in range(0,int(wordlen)):
        if(output[k] == trueVal[k]):
          cnt2char +=  1;
        else :
          mismatch_pair.append({output[k], trueVal[k]});
          strmismatch += output[k];
          tempcnt += 1;
          mismatchcnt += 1;
    if(output==trueVal):
      cnt = cnt+1; 
    ip = ""
    output = ""
    trueVal = ""
    avgmismatch += (tempcnt/wordlen);
  print(100*cnt/(int(l)))
  print(100*cnt2char/totalcharcnt);
  print(avgmismatch/mismatchcnt);
  print(set(strmismatch));
    

In [ ]:
mismatch_pair = [];
strmismatch = "";
predictAccuracy(encoder,decoder,maxlen_inp,maxlen_targ,mismatch_pair,strmismatch)

0.0 % completed
0.016220600162206 % completed
0.032441200324412 % completed
0.04866180048661801 % completed
0.064882400648824 % completed
0.08110300081103 % completed
0.09732360097323602 % completed
0.11354420113544202 % completed
0.129764801297648 % completed
0.145985401459854 % completed
0.16220600162206 % completed
0.17842660178426603 % completed
0.19464720194647203 % completed
0.21086780210867803 % completed
0.22708840227088403 % completed
0.24330900243309003 % completed
0.259529602595296 % completed
0.275750202757502 % completed
0.291970802919708 % completed
0.308191403081914 % completed
0.32441200324412 % completed
0.340632603406326 % completed
0.35685320356853206 % completed
0.37307380373073806 % completed
0.38929440389294406 % completed
0.40551500405515006 % completed
0.42173560421735606 % completed
0.43795620437956206 % completed
0.45417680454176806 % completed
0.47039740470397406 % completed
0.48661800486618007 % completed
0.5028386050283861 % completed
0.519059205190592 % co

In [ ]:
translate("$hamara@", encoder, decoder, maxlen_inp, maxlen_targ)

NameError: ignored

In [ ]:
translate("$bacche@", encoder, decoder, maxlen_inp, maxlen_targ)

$bacche@
Input: $bacche@
Predicted translation: बच्चे


In [ ]:
translate("$rahul@", encoder, decoder, maxlen_inp, maxlen_targ)

$rahul@
Input: $rahul@
Predicted translation: रहुल


In [ ]:
set1 = {}
set1[1]= 14
set1[4] = 11
set1[2] = 23

In [ ]:
set4 = {}
len(set4)

0

In [ ]:
len(set1)

3

In [ ]:
for i,j in set1.items():
  for id, val in enumerate(j): 
        print(val)
  print("sdfsf")

dsff
rahu
sdfsf
dsff
rahu
sdfsf
dsff
rahu
sdfsf


In [ ]:
for strng,probVal in sorted(set1.items(),  key = lambda kv:(kv[1], kv[0]),reverse=True):
  print(strng, probVal)

2 23
1 14
4 11


In [ ]:
for i in range(3):
  print(i)

0
1
2
